In [1]:
import networkx as nx
import pandas as pd
import numpy as np

In [14]:
edges = pd.read_csv(r'dataset\archive\edges.csv')
hero_network = pd.read_csv(r'dataset\archive\hero-network.csv')
nodes = pd.read_csv(r'dataset\archive\nodes.csv')